In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import random
import string
import tensorflow as tf

In [ ]:
with open('/content/name.tmp','r') as f:
  names = f.read().split("\n")[:-1]

In [ ]:
print("Number of Names: ",len(names))
print("\nMax Length of a Name: ",max(map(len,names))-1)

Number of Names:  3668

Max Length of a Name:  13


In [ ]:
names_size = len(names)
for i in range(10):
  a = random.randint(0, len(names))
  print(names[a])


Nield
Hodkinson
Ayton
Islam
Newcombe
Barry
Anley
Varley
Spiller
Ryan


In [ ]:
fig = ff.create_distplot([list(map(len,names))], group_labels=["Length"])
fig.update_layout(title="Name-Length Distribution")
fig.show()

In [ ]:
MAX_LENGTH = 10
names = [name for name in names if len(name)<=MAX_LENGTH]
print("Number of Names: ",len(names))

assert max(map(len,names))<=MAX_LENGTH, f"Names length more than {MAX_LENGTH}"

Number of Names:  3628


In [ ]:
start_token = " "

# we will use this pad_token to make size of each name equal
pad_token = "#"

#Add start token in front of all Names
names = [start_token+name for name in names]
MAX_LENGTH += 1

# set of tokens
tokens = sorted(set("".join(names + [pad_token])))

tokens = list(tokens)
n_tokens = len(tokens)
print("Tokens: ",tokens)
print ('n_tokens:', n_tokens)

Tokens:  [' ', '#', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
n_tokens: 53


In [ ]:
ttd = dict(zip(tokens,range(len(tokens))))

def to_matrix(names, mx_ln=None, pad=ttd[pad_token], dtype=np.int32):
  mx_ln = mx_ln or max(map(len, names))
  names_ix = np.zeros([len(names), mx_ln], dtype) + pad

  for i in range(len(names)):
      name_ix = list(map(ttd.get, names[i]))
      names_ix[i, :len(name_ix)] = name_ix

  return names_ix

In [ ]:
print('\n'.join(names[::200]))
print(to_matrix(names[::200]))

 Abbas
 Bailey
 Cain
 Daley
 Ealy
 Farnsworth
 Garrett
 Harden
 Irwin
 Kewley
 Lewis
 Mcgrory
 Norgrove
 Pemberton
 Renwick
 Simon
 Tilston
 Wadham
 Yarnall
[[ 0  2 28 28 27 45  1  1  1  1  1]
 [ 0  3 27 35 38 31 51  1  1  1  1]
 [ 0  4 27 35 40  1  1  1  1  1  1]
 [ 0  5 27 38 31 51  1  1  1  1  1]
 [ 0  6 27 38 51  1  1  1  1  1  1]
 [ 0  7 27 44 40 45 49 41 44 46 34]
 [ 0  8 27 44 44 31 46 46  1  1  1]
 [ 0  9 27 44 30 31 40  1  1  1  1]
 [ 0 10 44 49 35 40  1  1  1  1  1]
 [ 0 12 31 49 38 31 51  1  1  1  1]
 [ 0 13 31 49 35 45  1  1  1  1  1]
 [ 0 14 29 33 44 41 44 51  1  1  1]
 [ 0 15 41 44 33 44 41 48 31  1  1]
 [ 0 17 31 39 28 31 44 46 41 40  1]
 [ 0 19 31 40 49 35 29 37  1  1  1]
 [ 0 20 35 39 41 40  1  1  1  1  1]
 [ 0 21 35 38 45 46 41 40  1  1  1]
 [ 0 24 27 30 34 27 39  1  1  1  1]
 [ 0 25 27 44 40 27 38 38  1  1  1]]


In [ ]:
X = to_matrix(names)
X_train = np.zeros((X.shape[0],X.shape[1],n_tokens),np.int32)
y_train = np.zeros((X.shape[0],X.shape[1],n_tokens),np.int32)

for i, name in enumerate(X):
  for j in range(MAX_LENGTH-1):
      X_train[i,j,name[j]] = 1
      y_train[i,j,name[j+1]] = 1
  X_train[i,MAX_LENGTH-1,name[MAX_LENGTH-1]] = 1
  y_train[i,MAX_LENGTH-1,ttd[pad_token]] = 1

In [ ]:
def make_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Embedding(n_tokens,
                                      16,input_length=MAX_LENGTH))
  model.add(tf.keras.layers.SimpleRNN(256,
                                      return_sequences=True,
                                      activation='elu'))
  model.add(tf.keras.layers.SimpleRNN(256,
                                      return_sequences=True,
                                      activation='elu'))
  model.add(tf.keras.layers.Dense(n_tokens,
                                  activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
model = make_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 16)            848       
                                                                 
 simple_rnn (SimpleRNN)      (None, 11, 256)           69888     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 11, 256)           131328    
                                                                 
 dense (Dense)               (None, 11, 53)            13621     
                                                                 
Total params: 215685 (842.52 KB)
Trainable params: 215685 (842.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
name_count = X.shape[0]
BATCH_SIZE = 32
STEPS_PER_EPOCH = np.ceil(name_count/BATCH_SIZE)

AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False
train_dataset = (
  tf.data.Dataset.from_tensor_slices((X,y_train))
  .shuffle(5000)
  .cache()
  .repeat()
  .batch(BATCH_SIZE)
  .prefetch(AUTO))

In [ ]:
class CyclicLR(tf.keras.callbacks.Callback):

  def __init__(self,base_learning_rate=1e-5,mx_learning_rate=1e-3,ss=10):
      super().__init__()

      self.base_learning_rate = base_learning_rate
      self.mx_learning_rate = mx_learning_rate
      self.ss = ss
      self.iterations = 0
      self.history = {}

  def clr(self):
      cycle = np.floor((1+self.iterations)/(2*self.ss))
      x = np.abs(self.iterations/self.ss - 2*cycle + 1)

      return self.base_learning_rate + (self.mx_learning_rate - self.base_learning_rate)*(np.maximum(0,1-x))

  def on_train_begin(self,logs={}):
      tf.keras.backend.set_value(self.model.optimizer.lr, self.base_learning_rate)

  def on_batch_end(self,batch,logs=None):
      logs = logs or {}

      self.iterations += 1

      self.history.setdefault('lr', []).append(tf.keras.backend.get_value(self.model.optimizer.lr))
      self.history.setdefault('iterations', []).append(self.iterations)

      for k, v in logs.items():
          self.history.setdefault(k, []).append(v)

      tf.keras.backend.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
#%%time
cyclicLR = CyclicLR(1e-4,1e-3,6000)
EPOCHS = 100
history = model.fit(train_dataset,steps_per_epoch=STEPS_PER_EPOCH,epochs=EPOCHS,callbacks=[cyclicLR])



Epoch 1/100
114/114 [==============================] - 7s 33ms/step - loss: 2.5722
Epoch 2/100
114/114 [==============================] - 4s 34ms/step - loss: 2.0850
Epoch 3/100
114/114 [==============================] - 6s 48ms/step - loss: 2.0082
Epoch 4/100
114/114 [==============================] - 4s 34ms/step - loss: 1.9520
Epoch 5/100
114/114 [==============================] - 4s 33ms/step - loss: 1.8973
Epoch 6/100
114/114 [==============================] - 5s 45ms/step - loss: 1.8461
Epoch 7/100
114/114 [==============================] - 4s 33ms/step - loss: 1.7983
Epoch 8/100
114/114 [==============================] - 4s 33ms/step - loss: 1.7558
Epoch 9/100
114/114 [==============================] - 5s 45ms/step - loss: 1.7173
Epoch 10/100
114/114 [==============================] - 4s 32ms/step - loss: 1.6868
Epoch 11/100
114/114 [==============================] - 4s 33ms/step - loss: 1.6613
Epoch 12/100
114/114 [==============================] - 5s 43ms/step - loss: 1.6418
E

In [ ]:
plot = go.Figure()
plot.add_trace(go.Scatter(x=np.arange(1,len(history.history['loss'])+1), y=history.history['loss'], mode='lines+markers', name='Training loss'))
plot.update_layout(title_text="Training loss")
plot.show()

In [ ]:
def generateName(model=model, sp=start_token, mxl=MAX_LENGTH):
  assert len(sp)<mxl, f"Length of the Seed-phrase is more than Max-Length: {mxl}"
  name = [sp]
  x = np.zeros((1,mxl),np.int32)
  x[0,0:len(sp)] = [ttd[token] for token in sp]
  for i in range(len(sp),mxl):
      p = list(model.predict(x)[0,i-1])
      p = p/np.sum(p)
      index = np.random.choice(range(n_tokens),p=p)
      if index == ttd[pad_token]:
          break
      x[0,i] = index
      name.append(tokens[index])
  return "".join(name)

In [ ]:
weights = 'IndianNamesWeights.h5'
model.save_weights(weights)

In [ ]:
predictor = make_model()
predictor.load_weights(weights)

In [ ]:
sp = f" {np.random.choice(list(string.ascii_uppercase))}"
for _ in range(20):
  name = generateName(predictor,sp=sp)
  if name not in names:
      print(f"{name.lstrip()} (New Name)")
  else:
      print(name.lstrip())

1/1 [==============================] - 0s 21ms/step
Impleton (New Name)
1/1 [==============================] - 0s 23ms/step
Ingnall (New Name)
1/1 [==============================] - 0s 32ms/step
Istridfe (New Name)
1/1 [==============================] - 0s 23ms/step
Illett (New Name)
1/1 [==============================] - 0s 23ms/step
Idson (New Name)
1/1 [==============================] - 0s 22ms/step
Iril (New Name)
1/1 [==============================] - 0s 23ms/step
Ildyen (New Name)
1/1 [==============================] - 0s 26ms/step
Igwarn (New Name)
1/1 [==============================] - 0s 22ms/step
Iyger (New Name)
1/1 [==============================] - 0s 26ms/step
Idhrowe (New Name)
1/1 [==============================] - 0s 25ms/step
Irob (New Name)
1/1 [==============================] - 0s 24ms/step
Idson (New Name)
1/1 [==============================] - 0s 25ms/step
Ilthaw (New Name)
1/1 [==============================] - 0s 26ms/step
Ilbam (New Name)
1/1 [=================